Osnabrück University - Machine Learning (Summer Term 2016) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack

# Exercise Sheet 07

## Introduction

This week's sheet should be solved and handed in before the end of **Sunday, June 05, 2016**. If you need help (and Google and other resources were not enough), feel free to contact your groups designated tutor or whomever of us you run into first. Please upload your results to your group's studip folder.

## Assignment 1: The Perceptron [6 Points]

### a) The Logic Perceptron

For the following two logical functions sketch a perceptron's weights after it was trained. To do so, figure out when the perceptron should fire. Then come up with ideas of how you can achieve this. Remember that $w_0$, the bias, is used as a threshold and that we set a constant $x_0 = 1$.

#### 1) $(A \wedge B) \vee (\neg A \wedge B)$

The given function evaluates as follows:

| $A$ | $B$ | $A \wedge B$ | $\neg A \wedge B$ | $(A \wedge B) \vee (\neg A \wedge B)$ |
|-----|-----|--------------|-------------------|---------------------------------------|
|  0  |  0  |      0       |         0         |                 0                     |
|  0  |  1  |      0       |         1         |                 1                     |
|  1  |  0  |      0       |         0         |                 0                     |
|  1  |  1  |      1       |         0         |                 1                     |

Thus the function evaluates to true only if $B$ is true.

The perceptron activates if:

$$w_0 x_0 + w_1 x_1 + w_2 x_2 > 0$$

If $x_1$ is $A$ we can ignore it for example by setting $w_1 = 0$. To count $B$ we can adjust the weight $w_2 = 1$. So now we get the following activation function evaluations:

$$\begin{align*}
x = (0, 0) &: \quad 1 w_0 + 0 w_1 + 0 w_2 = 0 + w_0 \\
x = (0, 1) &: \quad 1 w_0 + 0 w_1 + 1 w_2 = 1 + w_0 \\
x = (1, 0) &: \quad 1 w_0 + 1 w_1 + 0 w_2 = 0 + w_0 \\
x = (1, 1) &: \quad 1 w_0 + 1 w_1 + 1 w_2 = 1 + w_0
\end{align*}$$

To fire on $B$ we can set the bias to $w_0 = -0.5$ such that for $x = (0, 0)$ and $x = (1, 0)$ the activation functions evaluate to values $\leq 0$ and in the other cases $> 0$.

#### 2) $(A \wedge B) \vee (\neg A \wedge B) \vee (A \wedge \neg B)$

The given function evaluates as follows:

| $A$ | $B$ | $A \wedge B$ | $\neg A \wedge B$ | $A \wedge \neg B$ | $(A \wedge B) \vee (\neg A \wedge B) \vee (A \wedge \neg B)$ |
|-----|-----|--------------|-------------------|-------------------|-----------|
|  0  |  0  |      0       |         0         |         0         |     0     |
|  0  |  1  |      0       |         1         |         0         |     1     |
|  1  |  0  |      0       |         0         |         1         |     1     |
|  1  |  1  |      1       |         0         |         0         |     1     |

Thus the function evaluates to true any of the variables $A$ or $B$ is true, but false if both are false.

The perceptron activates if:

$$w_0 x_0 + w_1 x_1 + w_2 x_2 > 0$$

If $x_1$ is $A$ we can set $w_1 = 1$. To count $B$ we can adjust the weight $w_2 = 1$. So now we get the following activation function evaluations:

$$\begin{align*}
x = (0, 0) &: \quad 1 w_0 + 0 w_1 + 0 w_2 = 0 + w_0 \\
x = (0, 1) &: \quad 1 w_0 + 0 w_1 + 1 w_2 = 1 + w_0 \\
x = (1, 0) &: \quad 1 w_0 + 1 w_1 + 0 w_2 = 1 + w_0 \\
x = (1, 1) &: \quad 1 w_0 + 1 w_1 + 1 w_2 = 2 + w_0
\end{align*}$$

To fire on $B$ we can set the bias to $w_0 = 0$ such that for $x = (0, 0)$ the activation function evaluates to $\leq 0$ and in the other cases $> 0$.

### b) The Tensorflow Perceptron

With the online tool [TensorFlow playground](https://playground.tensorflow.org/) it is possible to visual simple neural networks and to share configurations and settings. 

Follow [this link](https://playground.tensorflow.org/#activation=sigmoid&batchSize=1&dataset=gauss&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=1&seed=0.56339&showTestData=true&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&learningRate_hide=false&regularizationRate_hide=true&percTrainData_hide=true&batchSize_hide=true&dataset_hide=false&regularization_hide=true&discretize_hide=true&stepButton_hide=false&showTestData_hide=false&problem_hide=true&noise_hide=true&activation_hide=true) to the TensorFlow playground. If you click it, many features are disabled and set to useful defaults, since they were either not discussed yet in the lecture or are not important for this exercise.

You will see a simple configuration: Two activated inputs ($x_1$ and $x_2$), one hidden layer with one neuron (which can be understood as a simple perceptron) and the output shown as a nice picture. It initially shows a training loss of 0.527. Try and run it to see how the perceptron can learn to separate the two clusters. Note that for the rest of the exercise we assume at most about 1000 learning steps (usually many fewer will do it), so don't wait too long in front of your browser.

The dataset gets fully classified after very few iterations. Next try the XOR dataset, either by clicking on it on the left (the top right data pattern) or by following [this link](https://playground.tensorflow.org/#activation=sigmoid&batchSize=1&dataset=xor&regDataset=reg-plane&learningRate=0.1&regularizationRate=0&noise=0&networkShape=1&seed=0.56339&showTestData=true&discretize=false&percTrainData=50&x=true&y=true&xTimesY=false&xSquared=false&ySquared=false&cosX=false&sinX=false&cosY=false&sinY=false&collectStats=false&problem=classification&initZero=false&learningRate_hide=false&regularizationRate_hide=true&percTrainData_hide=true&batchSize_hide=true&dataset_hide=false&regularization_hide=true&discretize_hide=true&stepButton_hide=false&showTestData_hide=false&problem_hide=true&noise_hide=true&activation_hide=true). You will notice that you won't achieve much better results than a loss of 0.4, which is just above chance. Try to improve the result by adding neurons and or layers (but don't change the inputs!) until you get a classification with a loss smaller than 0.01. You may also change the learning rate. Then copy the link from your browser address bar and paste it below:

How many neurons in hidden layers are already sufficient to get at least 99% classification (i.e. loss < 0.01) if they are a) in one hidden layer or b) in two hidden layers? You may consider configurations which just need above 1000 iterations to get there as well, but we don't expect you to run any configuration longer than 1000 iterations.

## Assignment 2: Multilayer Perceptron [8 Points]

## Assignment 3: Sigmoid Activation & Backpropagation Delta Functions [6 Points]

In this exercise we are first going to take the derivative of a famous activation function - the sigmoid function
$$\sigma(t)=\frac{1}{1+e^{-t}}$$.
This function is commonly used because of its nice analytical properties: it domain is $\in[0,1]$, it is non-linear, strictly monotonous, continuous, differentiable and the derivative can be expressed in terms of the original function at the given point. This allows us to avoid redundant calculations. The sigmoid function is a special case of the more general *Logistic function* which can be found in many different fields: Biology, chemistry, economics, demography and recently most prominently: artificial neural networks.

Take the derivative $\frac{\partial \sigma}{\partial t}$ and (if possible) write the resulting expression in terms of $\sigma(t)$:

$$\begin{align}
\frac{\partial \sigma}{\partial t}&=-e^{-t} \cdot (-1) \cdot \frac{1}{(1+e^{-t})^2}\\
&= \frac{1}{1+e^{-t}} \cdot \frac{- 1 + 1 + e^{-t}}{1+e^{-t}} \\
&= \frac{1}{1+e^{-t}} \cdot \left(1 - \frac{1}{1+e^{-t}}\right) \\
&= \sigma(t)(1-\sigma(t))
\end{align}$$

Multilayer perceptrons (MLPs) can be regarded as a simple concatenation (and parallelization) of several perceptrons, each having a specified activation function $\sigma$ and a set of weights $\mathbf{w}_{ij}$. The idea that this can be done was discovered early after the invention of the perceptron, but people didn't really use it in practice because nobody really knew how to figure out the appropriate $\mathbf{w}_{ij}$. The solution to this problem was the discovery of the backpropagation algorithm which consists of two steps: first propagating the input forward through the layers of the MLP and storing the intermediate results and then propagating the error backwards and adjusting the weights of the units accordingly.

An updating rule for the output layer can be derived straightforward. The rules for the intermediate layers can be derived very similarly and only require a slight shift in perspective - the mathematics for that are however not in the standard toolkit so we are going to omit the calculations and refer you to the lecture slides.

We take the least-squares approach to derive the updating rule, i.e. we want to minimize the Loss function
$$L = \frac{1}{2}(y-t)^2$$
where t is the given (true) label from the dataset and y is the (single) output produced by the MLP. To find the weights that minimize this expression we want to take the derivative of $L$ w.r.t. $\mathbf{w}_{i}$ where we are now going to assume that the $\mathbf{w}_{i}$ are the ones directly before the output layer:
$$y = \sigma\left(\sum_{k=1}^n \mathbf{w}_{k}o_k\right)$$
Calculate $\frac{\partial L}{\partial \mathbf{w}_{i}}$.

*Hint*: Start here if you don't know what to do: $\frac{\partial L}{\partial \mathbf{w}_{i}} = \frac{\partial L}{\partial y}\frac{\partial y}{\partial \mathbf{w}_{i}}$

$$\begin{align}
\frac{\partial L}{\partial \mathbf{w}_{i}} &= \frac{\partial L}{\partial y}\frac{\partial y}{\partial \mathbf{w}_{i}}\\
&=(y-t)o_i\sigma\left(\sum_{k=1}^n \mathbf{w}_{k}o_k\right)\left(1-\sigma\left(\sum_{k=1}^n \mathbf{w}_{k}o_k\right)\right)\\
&= (y-t)o_iy\left(1-y\right)\\
\end{align}$$